In [1]:
import os
import yaml
import argparse
import numpy as np
import dask, dask_cudf
from dask_cuda import LocalCUDACluster
from dask.delayed import delayed
from dask.distributed import Client, wait, LocalCluster
from dask.utils import parse_bytes
import tritonclient.http as triton_http
import tritonclient.grpc as triton_grpc

import math
from math import cos, sin, asin, sqrt, pi

def jfk_distance(dropoff_latitude, dropoff_longitude, jfk_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_jfk = pi/180 * 40.6413
        y_jfk = pi/180 * -73.7781
        
        dlon = y_jfk - y_1
        dlat = x_jfk - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_jfk) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        jfk_distance[i] = c * r
        
def lga_distance(dropoff_latitude, dropoff_longitude, lga_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_lga = pi/180 * 40.7769
        y_lga = pi/180 * -73.8740
        
        dlon = y_lga - y_1
        dlat = x_lga - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_lga) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        lga_distance[i] = c * r
        
def ewr_distance(dropoff_latitude, dropoff_longitude, ewr_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_ewr = pi/180 * 40.6895
        y_ewr = pi/180 * -74.1745
        
        dlon = y_ewr - y_1
        dlat = x_ewr - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_ewr) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        ewr_distance[i] = c * r
        
def tsq_distance(dropoff_latitude, dropoff_longitude, tsq_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_tsq = pi/180 * 40.7580
        y_tsq = pi/180 * -73.9855
        
        dlon = y_tsq - y_1
        dlat = x_tsq - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_tsq) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        tsq_distance[i] = c * r
        
def met_distance(dropoff_latitude, dropoff_longitude, met_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_met = pi/180 * 40.7794
        y_met = pi/180 * -73.9632
        
        dlon = y_met - y_1
        dlat = x_met - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_met) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        met_distance[i] = c * r
        
def wtc_distance(dropoff_latitude, dropoff_longitude, wtc_distance):
    for i, (x_1, y_1) in enumerate(zip(dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_wtc = pi/180 * 40.7126
        y_wtc = pi/180 * -74.0099
        
        dlon = y_wtc - y_1
        dlat = x_wtc - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_wtc) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        wtc_distance[i] = c * r

def add_features(df):
    df['hour'] = df['pickup_datetime'].dt.hour
    df['year'] = df['pickup_datetime'].dt.year
    df['month'] = df['pickup_datetime'].dt.month
    df['day'] = df['pickup_datetime'].dt.day
    df['weekday'] = df['pickup_datetime'].dt.weekday
    
    df = df.apply_rows(jfk_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(jfk_distance=np.float32), kwargs=dict())
    
    df = df.apply_rows(lga_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(lga_distance=np.float32), kwargs=dict())
        
    df = df.apply_rows(ewr_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(ewr_distance=np.float32), kwargs=dict())
            
    df = df.apply_rows(tsq_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(tsq_distance=np.float32), kwargs=dict())
    
    df = df.apply_rows(met_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(met_distance=np.float32), kwargs=dict())
    
    df = df.apply_rows(wtc_distance, incols=['dropoff_latitude', 'dropoff_longitude'],
                       outcols=dict(wtc_distance=np.float32), kwargs=dict())
    
    df = df.drop(['pickup_datetime','key'], axis=1)
    
    return df

def triton_inference():
    http_client = triton_http.InferenceServerClient(
        url="13.92.226.55:8000",
        verbose=True,
        concurrency=12
    )
    grpc_client = triton_grpc.InferenceServerClient(
        url="13.92.226.55:8001",
        verbose = True
    )

    test = dask_cudf.read_csv('/home/nvidiatest/mlops_blog/data/test.csv')
    test = test.drop(['fare_amount'], axis=1)
    test = test.sample(frac=0.000001)

    test['key'] = test['key'].astype('datetime64[ns]')
    test['pickup_datetime'] = test['pickup_datetime'].astype('datetime64[ns]')
    test['pickup_longitude'] = test ['pickup_longitude'].astype('float32')
    test['pickup_latitude'] = test ['pickup_latitude'].astype('float32')
    test['dropoff_longitude'] = test ['dropoff_longitude'].astype('float32')
    test['dropoff_latitude'] = test ['dropoff_latitude'].astype('float32')
    test['passenger_count'] = test ['passenger_count'].astype('uint8')

    # now add the features
    tparts = [dask.delayed(add_features)(part) for part in test.to_delayed()]
    test = dask_cudf.from_delayed(tparts)
    test = test.astype('float32')
    test = test.compute().to_pandas().values

    # Set up Triton input and output objects for both HTTP and GRPC
    triton_input_http = triton_http.InferInput(
        'input__0',
        (test.shape[0], test.shape[1]),
        'FP32'
    )
    triton_input_http.set_data_from_numpy(test, binary_data=False)
    triton_output_http = triton_http.InferRequestedOutput(
        'output__0',
        binary_data=False
    )
    triton_input_grpc = triton_grpc.InferInput(
        'input__0',
        (test.shape[0], test.shape[1]),
        'FP32'
    )
    triton_input_grpc.set_data_from_numpy(test)
    triton_output_grpc = triton_grpc.InferRequestedOutput('output__0')

    request_http = http_client.infer(
    'fil',
    model_version='1',
    inputs=[triton_input_http],
    outputs=[triton_output_http]
    )

    request_grpc = grpc_client.infer(
        'fil',
        model_version='1',
        inputs=[triton_input_grpc],
        outputs=[triton_output_grpc]
    )

    # Get results as numpy arrays
    result_http = request_http.as_numpy('output__0')
    result_grpc = request_grpc.as_numpy('output__0')

    # Check that we got the same result with both GRPC and HTTP
    np.testing.assert_almost_equal(result_http, result_grpc)

In [2]:
triton_inference()

POST /v2/models/fil/versions/1/infer, headers None
{"inputs":[{"name":"input__0","shape":[6,16],"datatype":"FP32","data":[-73.9756088256836,40.7603874206543,-73.98173522949219,40.752479553222656,2.0,9.0,2015.0,6.0,5.0,4.0,21.155445098876953,9.470947265625,17.69024085998535,0.6909160017967224,3.375976800918579,5.0294904708862305,-73.98548126220703,40.74746322631836,-73.97423553466797,40.74087142944336,1.0,8.0,2009.0,1.0,28.0,2.0,19.900842666625977,9.34476089477539,17.818981170654297,2.127897024154663,4.383846759796143,4.349079132080078,-73.98924255371094,40.747642517089844,-73.95941162109375,40.76703643798828,1.0,22.0,2010.0,10.0,31.0,6.0,20.713951110839844,7.275616645812988,20.070541381835938,2.41605806350708,1.4112931489944458,7.398182392120361,-74.00364685058594,40.74864959716797,-74.00128936767578,40.736244201660156,2.0,14.0,2015.0,1.0,10.0,5.0,21.57731819152832,11.635576248168945,15.49663257598877,2.760690212249756,5.772342681884766,2.727410078048706,-73.99055480957031,40.761356353